Copie du programme Nettoyage données affluence par jour + nombre de validation par jour par station

In [35]:
import pandas as pd
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = '/workspaces/metro-JO/Données importées/2022_S2_PROFIL_FER.csv'
données_validation = pd.read_csv(url, delimiter=';')

# %%
# Convertir la colonne 'pourc_validations' en float (remplacer la virgule par un point pour séparer les décimales)
données_validation['pourc_validations'] = données_validation['pourc_validations'].str.replace(',', '.').astype(float)

# Grouper par 'LIBELLE_ARRET' et 'TRNC_HORR_60', puis calculer la somme des pourc_validations pour chaque groupe
tableau_somme_pour_validations = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()


# %%
# Mapping des tranches horaires à des heures spécifiques
tranche_horaire_to_hour = {
    '0H-1H': '00:00-01:00',
    '1H-2H': '01:00-02:00',
    '2H-3H': '02:00-03:00',
    '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00',
    '5H-6H': '05:00-06:00',
    '6H-7H': '06:00-07:00',
    '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00',
    '9H-10H': '09:00-10:00',
    '10H-11H': '10:00-11:00',
    '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00',
    '13H-14H': '13:00-14:00',
    '14H-15H': '14:00-15:00',
    '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00',
    '17H-18H': '17:00-18:00',
    '18H-19H': '18:00-19:00',
    '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00',
    '21H-22H': '21:00-22:00',
    '22H-23H': '22:00-23:00',
    '23H-24H': '23:00-00:00'
}

# Appliquer la conversion des tranches horaires au format heure
tableau_somme_pour_validations['TRNC_HORR_60'] = tableau_somme_pour_validations['TRNC_HORR_60'].map(tranche_horaire_to_hour)

# %%
# Regroupement des données en cas de doublons pour une paire 'LIBELLE_ARRET'-'TRNC_HORR_60'
grouped_data = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()

# Pivotement des données
pivot_table = grouped_data.pivot(index='LIBELLE_ARRET', columns='TRNC_HORR_60', values='pourc_validations').reset_index()

# Renommage des colonnes
pivot_table.columns.name = None  # Supprimer le nom de la colonne index
pivot_table = pivot_table.rename(columns=lambda x: f"{x}" if x != 'LIBELLE_ARRET' else x)

# %%
import re

# Sélectionner les colonnes correspondant aux tranches horaires
horaire_columns = pivot_table.columns[1:]  # Exclure la colonne 'LIBELLE_ARRET'

# Fonction pour extraire l'heure de début de chaque tranche horaire
def extract_start_hour(column):
    match = re.match(r'(\d+)H-(\d+)H', column)
    if match:
        return int(match.group(1))
    return 0

# Trier les colonnes dans l'ordre croissant des heures de début des tranches
sorted_columns = sorted(horaire_columns, key=extract_start_hour)

# Réorganiser le tableau pivoté avec les colonnes dans l'ordre chronologique
new_column_order = ['LIBELLE_ARRET'] + sorted_columns
pivot_table = pivot_table[new_column_order]

# Supprimer la colonne 'ND' du DataFrame pivot_table_ordered
pivot_table.drop(columns=['ND'], inplace=True, errors='ignore')

pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.strip()  # Supprime les espaces en début et fin
pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.lower()  # Met tout en minuscules

# Créer un dictionnaire de correspondance pour renommer les colonnes
correspondance_heures = {
    '0H-1H': '00:00-01:00', '1H-2H': '01:00-02:00', '2H-3H': '02:00-03:00', '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00', '5H-6H': '05:00-06:00', '6H-7H': '06:00-07:00', '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00', '9H-10H': '09:00-10:00', '10H-11H': '10:00-11:00', '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00', '13H-14H': '13:00-14:00', '14H-15H': '14:00-15:00', '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00', '17H-18H': '17:00-18:00', '18H-19H': '18:00-19:00', '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00', '21H-22H': '21:00-22:00', '22H-23H': '22:00-23:00', '23H-0H': '23:00-00:00'
}

# Renommer les colonnes en utilisant le dictionnaire de correspondance
pivot_table.rename(columns=correspondance_heures, inplace=True)

# Afficher le DataFrame après suppression de la colonne 'ND'
print(pivot_table)

       LIBELLE_ARRET  00:00-01:00  01:00-02:00  02:00-03:00  03:00-04:00  \
0           abbesses        10.18         3.20         0.02         0.00   
1              ablon         0.26         0.03         0.00         0.02   
2      acheres grand         0.78          NaN          NaN          NaN   
3      acheres ville         0.76         0.31         0.13         0.09   
4      aeroport cdg1         0.76         0.14         0.01         0.04   
..               ...          ...          ...          ...          ...   
751           vosves          NaN          NaN          NaN          NaN   
752  vulaines sur se         0.11          NaN          NaN          NaN   
753           wagram         3.63         1.43         0.10         0.00   
754           yerres         0.73         0.11         0.01         0.05   
755   yvris noisy gd         0.20         0.04          NaN          NaN   

     04:00-05:00  05:00-06:00  06:00-07:00  07:00-08:00  08:00-09:00  ...  \
0         

/tmp/ipykernel_2010/1933373044.py:4: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  données_validation = pd.read_csv(url, delimiter=';')


Création d'un dictionnaire contenant les fréquentations de chaque station de chaque ligne

In [36]:
import pandas as pd
import numpy as np

tableau_fréquentation = pd.read_excel('https://data.ratp.fr/api/explore/v2.1/catalog/datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2021/exports/xlsx?lang=fr&timezone=Europe%2FBerlin&use_labels=true')

print(tableau_fréquentation)

     Rang Réseau               Station    Trafic Correspondance_1  \
0       3  Métro          GARE DE LYON  28640475                1   
1      12  Métro            BELLEVILLE   7314438                2   
2      32  Métro                 OPERA   5193831                3   
3      58  Métro  SAINT-MANDE-TOURELLE   3944640                1   
4      59  Métro                 HOCHE   3928404                5   
..    ...    ...                   ...       ...              ...   
366    39    RER    FONTENAY-SOUS-BOIS   2121720                A   
367    40    RER         SUCY-BONNEUIL   1993503                A   
368    58    RER            LE GUICHET    530706                B   
369    65    RER        PARC DE SCEAUX    383626                B   
370    66    RER        LA HACQUINIERE    285626                B   

    Correspondance_2 Correspondance_3  Correspondance_4  Correspondance_5  \
0                 14                A               NaN               NaN   
1                

In [37]:
tableau_fréquentation = tableau_fréquentation[tableau_fréquentation['Réseau'] == 'Métro']
display(tableau_fréquentation)

,Rang,Réseau,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,Ville,Arrondissement pour Paris
0,3,Métro,GARE DE LYON,28640475,1,14,A,NaN,NaN,Paris,12.0
1,12,Métro,BELLEVILLE,7314438,2,11,NaN,NaN,NaN,Paris,10.0
2,32,Métro,OPERA,5193831,3,7,8,NaN,NaN,Paris,9.0
3,58,Métro,SAINT-MANDE-TOURELLE,3944640,1,NaN,NaN,NaN,NaN,Saint Mandé,NaN
4,59,Métro,HOCHE,3928404,5,NaN,NaN,NaN,NaN,Pantin,NaN
...,...,...,...,...,...,...,...,...,...,...,...
350,272,Métro,SULLY-MORLAND,1124169,7,NaN,NaN,NaN,NaN,Paris,4.0
351,275,Métro,EUROPE,1106459,3,NaN,NaN,NaN,NaN,Paris,8.0
352,283,Métro,CAMPO-FORMIO,1016150,5,NaN,NaN,NaN,NaN,Paris,13.0
353,298,Métro,SAINT-FARGEAU,472258,3bis,NaN,NaN,NaN,NaN,Paris,8.0


In [38]:
colonnes_a_supprimer = ['Réseau', 'Ville', 'Arrondissement pour Paris']

tableau_fréquentation = tableau_fréquentation.drop(colonnes_a_supprimer, axis=1)

# Afficher le DataFrame après suppression des colonnes
print(tableau_fréquentation)

     Rang               Station    Trafic Correspondance_1 Correspondance_2  \
0       3          GARE DE LYON  28640475                1               14   
1      12            BELLEVILLE   7314438                2               11   
2      32                 OPERA   5193831                3                7   
3      58  SAINT-MANDE-TOURELLE   3944640                1              NaN   
4      59                 HOCHE   3928404                5              NaN   
..    ...                   ...       ...              ...              ...   
350   272         SULLY-MORLAND   1124169                7              NaN   
351   275                EUROPE   1106459                3              NaN   
352   283          CAMPO-FORMIO   1016150                5              NaN   
353   298         SAINT-FARGEAU    472258             3bis              NaN   
354   300       PORTE D'AUTEUIL    375748               10              NaN   

    Correspondance_3  Correspondance_4  Corresponda

In [39]:
# Liste des colonnes à modifier
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Convertir les colonnes spécifiées en type chaîne (si elles ne le sont pas déjà)
tableau_fréquentation[colonnes_correspondance] = tableau_fréquentation[colonnes_correspondance].astype(str)

# Supprimer les lettres 'A' et 'B' des colonnes spécifiées dans tableau_fréquentation
for colonne in colonnes_correspondance:
    tableau_fréquentation[colonne] = tableau_fréquentation[colonne].str.replace('A', '').str.replace('B', '')

# Afficher le DataFrame après suppression des lettres spécifiées
print(tableau_fréquentation)

     Rang               Station    Trafic Correspondance_1 Correspondance_2  \
0       3          GARE DE LYON  28640475                1               14   
1      12            BELLEVILLE   7314438                2               11   
2      32                 OPERA   5193831                3                7   
3      58  SAINT-MANDE-TOURELLE   3944640                1              nan   
4      59                 HOCHE   3928404                5              nan   
..    ...                   ...       ...              ...              ...   
350   272         SULLY-MORLAND   1124169                7              nan   
351   275                EUROPE   1106459                3              nan   
352   283          CAMPO-FORMIO   1016150                5              nan   
353   298         SAINT-FARGEAU    472258             3bis              nan   
354   300       PORTE D'AUTEUIL    375748               10              nan   

    Correspondance_3 Correspondance_4 Correspondanc

In [40]:
# Liste des colonnes à modifier
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Remplacer "nan" par "NaN" dans les colonnes spécifiées
tableau_fréquentation[colonnes_correspondance] = tableau_fréquentation[colonnes_correspondance].replace('nan', 'NaN')

# Afficher le DataFrame avec les remplacements effectués
print(tableau_fréquentation)

     Rang               Station    Trafic Correspondance_1 Correspondance_2  \
0       3          GARE DE LYON  28640475                1               14   
1      12            BELLEVILLE   7314438                2               11   
2      32                 OPERA   5193831                3                7   
3      58  SAINT-MANDE-TOURELLE   3944640                1              NaN   
4      59                 HOCHE   3928404                5              NaN   
..    ...                   ...       ...              ...              ...   
350   272         SULLY-MORLAND   1124169                7              NaN   
351   275                EUROPE   1106459                3              NaN   
352   283          CAMPO-FORMIO   1016150                5              NaN   
353   298         SAINT-FARGEAU    472258             3bis              NaN   
354   300       PORTE D'AUTEUIL    375748               10              NaN   

    Correspondance_3 Correspondance_4 Correspondanc

In [41]:
# Liste des colonnes à analyser
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Calcul du nombre de cases non vides pour chaque ligne
tableau_fréquentation['nb_correspondance'] = tableau_fréquentation[colonnes_correspondance].count(axis=1)

# Afficher le DataFrame avec la nouvelle colonne "nb_correspondance"
print(tableau_fréquentation)

     Rang               Station    Trafic Correspondance_1 Correspondance_2  \
0       3          GARE DE LYON  28640475                1               14   
1      12            BELLEVILLE   7314438                2               11   
2      32                 OPERA   5193831                3                7   
3      58  SAINT-MANDE-TOURELLE   3944640                1              NaN   
4      59                 HOCHE   3928404                5              NaN   
..    ...                   ...       ...              ...              ...   
350   272         SULLY-MORLAND   1124169                7              NaN   
351   275                EUROPE   1106459                3              NaN   
352   283          CAMPO-FORMIO   1016150                5              NaN   
353   298         SAINT-FARGEAU    472258             3bis              NaN   
354   300       PORTE D'AUTEUIL    375748               10              NaN   

    Correspondance_3 Correspondance_4 Correspondanc

In [42]:
# Diviser les valeurs de la colonne "Trafic" par les valeurs de la colonne "nb_correspondance"
tableau_fréquentation['Trafic'] = tableau_fréquentation['Trafic'] / tableau_fréquentation['nb_correspondance']

# Afficher le DataFrame avec les valeurs mises à jour
print(tableau_fréquentation)

     Rang               Station     Trafic Correspondance_1 Correspondance_2  \
0       3          GARE DE LYON  5728095.0                1               14   
1      12            BELLEVILLE  1462887.6                2               11   
2      32                 OPERA  1038766.2                3                7   
3      58  SAINT-MANDE-TOURELLE   788928.0                1              NaN   
4      59                 HOCHE   785680.8                5              NaN   
..    ...                   ...        ...              ...              ...   
350   272         SULLY-MORLAND   224833.8                7              NaN   
351   275                EUROPE   221291.8                3              NaN   
352   283          CAMPO-FORMIO   203230.0                5              NaN   
353   298         SAINT-FARGEAU    94451.6             3bis              NaN   
354   300       PORTE D'AUTEUIL    75149.6               10              NaN   

    Correspondance_3 Correspondance_4 C

In [43]:
#Nettoyage de données sur le nombre de validations: on obtient le nombre de validation moyen par jour sur 1 mois précis

import pandas as pd
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url2 = '/workspaces/metro-JO/Données importées/2022_S2_NB_FER.csv'
données_validation2= pd.read_csv(url2)
données_validation2 = pd.read_csv(url2, delimiter=';')

# Exemple avec gestion de formats flexibles
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], errors='coerce', format="%d/%m/%Y")
donnees_mai = données_validation2.loc[(données_validation2['JOUR'].dt.month == 5)]
donnees_sum = donnees_mai.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Conversion de la colonne 'JOUR' en datetime
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrer les données pour les dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Grouper par 'JOUR', 'LIBELLE_ARRET' et 'CATEGORIE_TITRE' et calculer la somme des 'NB_VALD'
donnees_aggreg = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET', 'CATEGORIE_TITRE'])['NB_VALD'].sum().reset_index()

# Conversion de la colonne 'JOUR' en datetime si ce n'est pas déjà fait
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrage des dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Somme de NB_VALD par JOUR et par LIBELLE_ARRET
somme_par_jour_arret = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET'])['NB_VALD'].sum().reset_index()

moyenne_par_arret = somme_par_jour_arret.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Normalisation des libellés
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.lower().str.strip()

# Somme des valeurs de NB_VALD pour les libellés normalisés
moyenne_par_arret = moyenne_par_arret.groupby('LIBELLE_ARRET')['NB_VALD'].sum().reset_index()

# Suppression des doublons
moyenne_par_arret = moyenne_par_arret.drop_duplicates(subset=['LIBELLE_ARRET'])

display(somme_par_jour_arret)
display(moyenne_par_arret)

,JOUR,LIBELLE_ARRET,NB_VALD
0,2022-07-24,ABBESSES,1867
1,2022-07-24,ABLON,600
2,2022-07-24,ACHERES GRAND,39
3,2022-07-24,ACHERES VILLE,3002
4,2022-07-24,AEROPORT CDG1,2955
...,...,...,...
13776,2022-08-11,VOSVES,20
13777,2022-08-11,VULAINES SUR SE,44
13778,2022-08-11,WAGRAM,3063
13779,2022-08-11,YERRES,2729


,LIBELLE_ARRET,NB_VALD
0,abbesses,2277.315789
1,ablon,1101.105263
2,acheres grand,51.388889
3,acheres ville,3012.052632
4,aeroport cdg1,5050.947368
...,...,...
733,vosves,14.894737
734,vulaines sur se,42.894737
735,wagram,3268.157895
736,yerres,2743.000000


In [44]:
import pandas as pd

# Vos données et numéro
# Assumons que vous avez déjà vos données dans un DataFrame nommé tableau_fréquentation
numéro = [1, 2, 3, '3bis', 4, 5, 6, 7, '7bis', 8, 9, 10, 11, 12, 13, 14]

# Création des tableaux pour chaque élément de numéro
tableaux = {}

colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

for k in numéro:
    # Convertir k en string car il peut contenir des valeurs non numériques
    k_str = str(k)
    
    # Filtrer les lignes où la valeur k est présente dans les colonnes spécifiées
    filtered_rows = tableau_fréquentation[tableau_fréquentation[colonnes_correspondance].apply(lambda x: k_str in x.values, axis=1)]
    
    # Stocker les lignes filtrées dans un tableau nommé "ligne numéro[k]"
    tableaux[f'ligne numéro[{k_str}]'] = filtered_rows

# Parcourir chaque tableau dans le dictionnaire et supprimer les colonnes spécifiées
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau.drop(columns=colonnes_correspondance)

# Parcourir chaque tableau dans le dictionnaire
for tableau_nom, tableau in tableaux.items():
    # Calculer la fréquentation totale pour chaque tableau
    fréquentation_totale_ligne = tableau['Trafic'].sum()

    # Ajouter la variable fréquentation_totale à chaque tableau
    tableaux[tableau_nom]['fréquentation_totale_ligne'] = fréquentation_totale_ligne
    tableau['Trafic_divisé'] = tableau['Trafic'] / tableau['fréquentation_totale_ligne']

# Supposons que la clé pour la correspondance soit 'LIBELLE_ARRET' dans le tableau moyenne_par_arret
cle_correspondance = 'LIBELLE_ARRET'

# Nettoyage des espaces et uniformisation de la casse
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.strip().str.lower()
for tableau_nom, tableau in tableaux.items():
    tableau['Station'] = tableau['Station'].str.strip().str.lower()

    # Fusion en utilisant les noms de stations nettoyés
    merged_table = pd.merge(tableau, moyenne_par_arret, left_on='Station', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD de moyenne_par_arret
    merged_table = merged_table.rename(columns={'NB_VALD': 'fréquentation moyenne hors JO'})
    
    tableaux[tableau_nom] = merged_table

Compléter avec les valeurs manquantes

In [45]:
import difflib

# Méthode pour trouver la correspondance la plus proche
def find_closest_match(target, candidates):
    closest_match = difflib.get_close_matches(target, candidates, n=1, cutoff=0.6)  # Ajustez le seuil selon vos besoins
    return closest_match[0] if closest_match else None

for k in numéro:
    k_str = str(k)
    # Accéder aux tableaux spécifiques
    current_table = tableaux[f'ligne numéro[{k_str}]']
    
    # Filtrer les lignes avec des valeurs manquantes dans la colonne 'LIBELLE_ARRET'
    missing_data = current_table[current_table['LIBELLE_ARRET'].isnull()]
    
    # Remplir les valeurs manquantes
    for index, row in missing_data.iterrows():
        closest_match = find_closest_match(row['Station'], moyenne_par_arret['LIBELLE_ARRET'])
        
        if closest_match:
            tableaux[f'ligne numéro[{k_str}]'].at[index, 'LIBELLE_ARRET'] = closest_match



In [46]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner avec le tableau moyenne_par_arret pour obtenir les valeurs NB_VALD associées
    merged_with_nb_vald = pd.merge(current_table, moyenne_par_arret[['LIBELLE_ARRET', 'NB_VALD']],
                                   left_on='LIBELLE_ARRET', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD pour plus de clarté
    merged_with_nb_vald = merged_with_nb_vald.rename(columns={'NB_VALD': 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'})

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = merged_with_nb_vald

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner les données avec le tableau contenant les nouvelles colonnes
    merged_table = pd.merge(current_table, pivot_table, on='LIBELLE_ARRET', how='left')

    # Mettre à jour le tableau dans le dictionnaire avec les nouvelles colonnes fusionnées
    tableaux[f'ligne numéro[{k_str}]'] = merged_table

In [47]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les colonnes 'fréquentation moyenne hors JO' et 'LIBELLE_ARRET' si elles existent
    columns_to_drop = ['fréquentation moyenne hors JO', 'LIBELLE_ARRET']
    current_table = current_table.drop(columns=columns_to_drop, errors='ignore')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table



In [48]:

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Colonnes à supprimer
    columns_to_drop = ['Trafic', 'fréquentation_totale_ligne', 'Trafic_divisé']

    # Vérifier si les colonnes existent avant de les supprimer
    columns_exist = [col for col in columns_to_drop if col in current_table.columns]

    # Supprimer les colonnes existantes
    if columns_exist:
        current_table = current_table.drop(columns=columns_exist)

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table

    # Afficher le tableau mis à jour
    print(f"Tableau 'ligne numéro[{k_str}]' sans les colonnes 'Trafic', 'fréquentation_totale_ligne', 'Trafic_divisé' :")
    display(tableaux[f'ligne numéro[{k_str}]'])


Tableau 'ligne numéro[1]' sans les colonnes 'Trafic', 'fréquentation_totale_ligne', 'Trafic_divisé' :


,Rang,Station,nb_correspondance,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,...,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00
0,3,gare de lyon,5,66037.947368,2.16,0.01,0.00,NaN,0.34,1.20,...,27.26,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80
1,3,gare de lyon,5,66037.947368,18.38,4.31,0.30,0.10,0.62,8.89,...,53.19,62.73,67.42,84.89,89.26,74.77,56.04,41.78,38.76,34.89
2,58,saint-mande-tourelle,5,8727.684211,3.59,1.06,0.10,0.00,0.01,3.76,...,34.11,35.69,37.54,44.98,47.84,34.26,19.04,10.38,8.05,5.99
3,192,louvre,5,4374.473684,14.09,4.62,0.10,0.00,0.02,1.16,...,29.11,38.64,47.98,59.32,63.31,49.76,28.05,19.06,19.83,20.34
4,25,porte de vincennes,5,12151.052632,3.90,1.27,0.10,0.00,0.00,5.48,...,33.49,35.67,38.04,40.51,38.45,30.06,18.82,12.32,9.06,6.66
5,51,porte maillot,5,7384.429150,2.64,0.07,0.02,0.00,0.03,1.79,...,26.10,34.32,45.86,68.87,69.45,51.21,31.13,17.93,11.92,7.54
6,51,porte maillot,5,7384.429150,10.76,3.99,0.11,0.00,0.00,3.90,...,29.56,34.51,35.90,44.22,44.75,35.65,22.26,15.20,16.55,18.25
7,71,chateau de vincennes,5,12151.052632,3.90,1.27,0.10,0.00,0.00,5.48,...,33.49,35.67,38.04,40.51,38.45,30.06,18.82,12.32,9.06,6.66
8,10,chatelet,5,87779.526316,30.17,10.60,0.27,0.04,0.36,7.19,...,55.16,72.79,92.19,114.31,122.78,105.60,70.85,51.84,47.85,44.60
9,41,pont de neuilly,5,10573.157895,3.28,1.22,0.13,0.03,0.02,2.50,...,34.15,36.23,39.03,46.88,51.98,39.18,21.86,11.94,9.25,6.68


Tableau 'ligne numéro[2]' sans les colonnes 'Trafic', 'fréquentation_totale_ligne', 'Trafic_divisé' :


,Rang,Station,nb_correspondance,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,...,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00
0,12,belleville,5,16611.631579,7.73,3.56,0.12,0.00,0.00,4.47,...,35.54,38.84,39.88,41.62,37.49,32.01,21.30,14.72,13.35,11.97
1,104,colonel fabien,5,6090.315789,7.43,3.17,0.16,0.00,0.02,3.23,...,32.24,35.77,39.02,45.10,41.33,34.81,22.89,15.25,12.20,10.86
2,118,menilmontant,5,5824.315789,13.64,6.06,0.25,NaN,0.00,5.44,...,30.90,32.69,34.55,37.85,37.05,33.65,24.10,18.15,17.53,17.79
3,163,blanche,5,4236.421053,16.80,7.78,0.19,0.00,0.03,5.05,...,29.47,34.19,37.46,41.29,41.39,36.88,27.88,21.15,27.09,24.68
4,256,philippe auguste,5,2473.842105,5.48,2.24,0.16,0.00,0.02,3.26,...,33.32,36.68,41.25,49.12,42.83,32.67,21.55,13.32,12.08,9.82
5,268,monceau,5,2529.789474,2.91,1.19,0.15,0.00,0.17,0.92,...,31.56,39.02,48.87,62.67,65.78,47.89,22.54,11.59,7.52,5.00
6,33,place clichy,5,10922.736842,13.84,5.13,0.13,0.00,0.06,3.44,...,30.26,34.37,37.23,41.68,40.48,38.04,26.06,20.67,21.62,21.26
7,38,la chapelle,5,13644.368421,4.04,1.16,0.00,0.00,0.00,4.10,...,34.17,37.97,43.28,49.12,47.07,40.78,31.32,21.51,13.47,8.41
8,52,jaures,5,11482.421053,11.44,4.55,0.18,0.24,0.31,4.65,...,30.37,34.15,37.88,44.07,41.76,36.84,24.98,20.83,16.90,15.14
9,249,avron,5,2578.736842,7.15,2.70,0.13,0.00,0.01,2.76,...,34.60,35.92,35.94,39.07,39.59,37.36,25.67,16.63,11.96,10.53


Tableau 'ligne numéro[3]' sans les colonnes 'Trafic', 'fréquentation_totale_ligne', 'Trafic_divisé' :


...

,Rang,Station,nb_correspondance,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,...,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00
0,242,rue du bac,5,2642.842105,7.98,2.21,0.04,0.00,0.00,0.92,...,33.95,40.07,50.13,57.77,57.82,48.87,23.13,15.46,12.09,11.93
1,288,rennes,5,1546.052632,4.45,1.57,0.08,0.00,0.09,1.01,...,38.12,44.08,52.01,58.20,56.48,44.05,23.63,10.44,10.24,8.72
2,4,montparnasse-bienvenue,5,67112.684211,5.71,0.26,0.01,0.04,0.24,3.15,...,22.86,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49
3,4,montparnasse-bienvenue,5,67112.684211,8.99,1.58,0.08,0.00,0.00,2.24,...,30.38,30.69,32.64,40.52,44.32,41.17,32.13,24.22,21.47,19.70
4,27,madeleine,5,12904.421053,4.97,1.52,0.11,0.02,0.00,0.87,...,31.86,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10
5,88,sevres-babylone,5,6540.105263,4.71,1.28,0.07,0.01,0.00,0.78,...,31.21,39.67,51.13,59.22,66.54,57.85,35.27,12.25,9.71,8.96
6,115,mairie d'issy,5,6616.526316,2.55,0.89,0.06,0.00,0.05,4.60,...,33.94,34.97,36.23,38.97,38.91,30.74,16.37,8.48,6.66,5.20
7,188,notre-dame-de-lorette,5,3949.578947,10.62,4.42,0.04,0.00,0.00,1.85,...,31.34,35.01,35.65,44.18,49.69,39.57,25.65,17.51,18.02,18.33
8,93,porte de versailles,5,15023.947368,12.16,4.87,0.09,0.01,0.00,1.58,...,28.53,39.11,50.70,60.08,62.00,53.19,32.02,19.18,19.01,18.49
9,136,vaugirard,5,4768.736842,4.38,1.34,0.07,0.00,0.00,1.95,...,35.49,37.27,41.28,44.22,41.04,34.89,19.16,10.52,9.18,7.66


Tableau 'ligne numéro[13]' sans les colonnes 'Trafic', 'fréquentation_totale_ligne', 'Trafic_divisé' :


,Rang,Station,nb_correspondance,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,...,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00
0,74,saint-denis-universite,5,8049.368421,1.95,0.42,0.01,0.00,0.00,18.07,...,28.65,30.33,28.90,31.51,27.77,18.64,13.60,8.89,6.17,4.36
1,122,les courtilles,5,8152.789474,2.39,0.58,0.02,0.00,0.00,11.63,...,30.01,30.03,31.76,35.24,31.76,23.70,16.18,10.77,7.34,4.88
2,125,porte de saint-ouen,5,6262.368421,5.48,2.05,0.05,0.00,0.00,4.58,...,32.40,34.06,38.01,42.89,40.38,34.53,23.47,16.62,12.24,9.27
3,29,porte de clichy,5,12615.105263,9.95,3.26,0.27,0.06,0.65,11.07,...,61.47,67.67,75.58,92.43,88.85,71.86,46.94,32.62,24.38,18.66
4,33,place clichy,5,10922.736842,13.84,5.13,0.13,0.00,0.06,3.44,...,30.26,34.37,37.23,41.68,40.48,38.04,26.06,20.67,21.62,21.26
5,87,porte de vanves,5,7711.315789,4.61,1.30,0.07,0.01,0.00,5.18,...,31.46,32.51,35.64,41.18,39.19,30.95,21.34,13.95,10.18,8.11
6,162,pernety,5,4404.263158,5.99,1.92,0.09,0.01,0.00,3.69,...,33.18,33.91,33.31,34.66,34.29,31.53,21.05,13.67,12.71,10.96
7,230,carrefour pleyel,5,3933.789474,3.40,1.46,0.10,0.00,0.05,4.69,...,30.52,33.97,48.79,57.68,45.30,28.87,16.22,10.19,8.39,6.78
8,64,gabriel peri,5,9837.842105,3.24,0.79,0.02,0.00,0.00,7.05,...,31.64,31.95,34.54,38.88,38.49,29.48,18.40,11.34,7.23,5.40
9,69,miromesnil,5,8603.789474,8.08,2.75,0.05,0.01,0.05,1.50,...,30.14,37.13,44.97,60.25,65.41,47.45,26.32,13.69,14.06,11.90


Tableau 'ligne numéro[14]' sans les colonnes 'Trafic', 'fréquentation_totale_ligne', 'Trafic_divisé' :


,Rang,Station,nb_correspondance,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,...,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00
0,3,gare de lyon,5,66037.947368,2.16,0.01,0.00,NaN,0.34,1.20,...,27.26,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80
1,3,gare de lyon,5,66037.947368,18.38,4.31,0.30,0.10,0.62,8.89,...,53.19,62.73,67.42,84.89,89.26,74.77,56.04,41.78,38.76,34.89
2,61,bercy,5,9023.947368,NaN,NaN,NaN,NaN,NaN,6.98,...,28.84,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09
3,61,bercy,5,9023.947368,10.59,2.39,0.22,0.02,0.30,3.90,...,24.50,25.89,33.22,43.36,44.68,36.17,35.22,24.11,25.58,36.50
4,27,madeleine,5,12904.421053,4.97,1.52,0.11,0.02,0.00,0.87,...,31.86,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10
5,29,porte de clichy,5,12615.105263,9.95,3.26,0.27,0.06,0.65,11.07,...,61.47,67.67,75.58,92.43,88.85,71.86,46.94,32.62,24.38,18.66
6,110,cour saint-emilion,5,3173.444444,11.21,1.93,0.12,0.00,0.00,1.45,...,28.48,36.59,49.66,61.54,60.91,45.62,29.16,25.69,15.63,12.76
7,6,bibliotheque,5,6088.000000,6.46,1.35,0.06,0.00,0.10,5.02,...,30.38,35.33,43.36,54.02,56.83,44.83,26.99,18.84,11.58,8.76
8,31,olympiades,5,12997.142857,2.74,1.23,0.06,0.00,0.01,6.71,...,35.48,38.44,42.90,47.26,41.98,31.78,20.50,13.19,6.90,4.47
9,39,mairie de saint-ouen,5,12017.736842,7.75,2.71,0.19,0.01,0.01,11.46,...,62.26,67.36,77.62,93.80,89.63,63.45,40.80,24.30,16.35,13.66


In [49]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance']

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance'] / 2

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si la colonne existe avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne' in current_table.columns:
        # Calculer la somme de la colonne spécifique
        current_table['nombre_validations_totales_pour_la_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'].sum()

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table
        # Afficher le tableau mis à jour
        print(f"Tableau 'ligne numéro[{k_str}]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :")
        display(tableaux[f'ligne numéro[{k_str}]'])


Tableau 'ligne numéro[1]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,nb_correspondance,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,3,gare de lyon,5,66037.947368,2.16,0.01,0.00,NaN,0.34,1.20,...,57.99,60.21,38.73,27.60,17.40,10.24,4.80,13207.589474,6603.794737,102578.37166
1,3,gare de lyon,5,66037.947368,18.38,4.31,0.30,0.10,0.62,8.89,...,84.89,89.26,74.77,56.04,41.78,38.76,34.89,13207.589474,6603.794737,102578.37166
2,58,saint-mande-tourelle,5,8727.684211,3.59,1.06,0.10,0.00,0.01,3.76,...,44.98,47.84,34.26,19.04,10.38,8.05,5.99,1745.536842,872.768421,102578.37166
3,192,louvre,5,4374.473684,14.09,4.62,0.10,0.00,0.02,1.16,...,59.32,63.31,49.76,28.05,19.06,19.83,20.34,874.894737,437.447368,102578.37166
4,25,porte de vincennes,5,12151.052632,3.90,1.27,0.10,0.00,0.00,5.48,...,40.51,38.45,30.06,18.82,12.32,9.06,6.66,2430.210526,1215.105263,102578.37166
5,51,porte maillot,5,7384.429150,2.64,0.07,0.02,0.00,0.03,1.79,...,68.87,69.45,51.21,31.13,17.93,11.92,7.54,1476.885830,738.442915,102578.37166
6,51,porte maillot,5,7384.429150,10.76,3.99,0.11,0.00,0.00,3.90,...,44.22,44.75,35.65,22.26,15.20,16.55,18.25,1476.885830,738.442915,102578.37166
7,71,chateau de vincennes,5,12151.052632,3.90,1.27,0.10,0.00,0.00,5.48,...,40.51,38.45,30.06,18.82,12.32,9.06,6.66,2430.210526,1215.105263,102578.37166
8,10,chatelet,5,87779.526316,30.17,10.60,0.27,0.04,0.36,7.19,...,114.31,122.78,105.60,70.85,51.84,47.85,44.60,17555.905263,8777.952632,102578.37166
9,41,pont de neuilly,5,10573.157895,3.28,1.22,0.13,0.03,0.02,2.50,...,46.88,51.98,39.18,21.86,11.94,9.25,6.68,2114.631579,1057.315789,102578.37166


Tableau 'ligne numéro[2]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,nb_correspondance,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,12,belleville,5,16611.631579,7.73,3.56,0.12,0.00,0.00,4.47,...,41.62,37.49,32.01,21.30,14.72,13.35,11.97,3322.326316,1661.163158,35036.421053
1,104,colonel fabien,5,6090.315789,7.43,3.17,0.16,0.00,0.02,3.23,...,45.10,41.33,34.81,22.89,15.25,12.20,10.86,1218.063158,609.031579,35036.421053
2,118,menilmontant,5,5824.315789,13.64,6.06,0.25,NaN,0.00,5.44,...,37.85,37.05,33.65,24.10,18.15,17.53,17.79,1164.863158,582.431579,35036.421053
3,163,blanche,5,4236.421053,16.80,7.78,0.19,0.00,0.03,5.05,...,41.29,41.39,36.88,27.88,21.15,27.09,24.68,847.284211,423.642105,35036.421053
4,256,philippe auguste,5,2473.842105,5.48,2.24,0.16,0.00,0.02,3.26,...,49.12,42.83,32.67,21.55,13.32,12.08,9.82,494.768421,247.384211,35036.421053
5,268,monceau,5,2529.789474,2.91,1.19,0.15,0.00,0.17,0.92,...,62.67,65.78,47.89,22.54,11.59,7.52,5.00,505.957895,252.978947,35036.421053
6,33,place clichy,5,10922.736842,13.84,5.13,0.13,0.00,0.06,3.44,...,41.68,40.48,38.04,26.06,20.67,21.62,21.26,2184.547368,1092.273684,35036.421053
7,38,la chapelle,5,13644.368421,4.04,1.16,0.00,0.00,0.00,4.10,...,49.12,47.07,40.78,31.32,21.51,13.47,8.41,2728.873684,1364.436842,35036.421053
8,52,jaures,5,11482.421053,11.44,4.55,0.18,0.24,0.31,4.65,...,44.07,41.76,36.84,24.98,20.83,16.90,15.14,2296.484211,1148.242105,35036.421053
9,249,avron,5,2578.736842,7.15,2.70,0.13,0.00,0.01,2.76,...,39.07,39.59,37.36,25.67,16.63,11.96,10.53,515.747368,257.873684,35036.421053


Tableau 'ligne numéro[3]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


...

,Rang,Station,nb_correspondance,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,242,rue du bac,5,2642.842105,7.98,2.21,0.04,0.00,0.00,0.92,...,57.77,57.82,48.87,23.13,15.46,12.09,11.93,528.568421,264.284211,104976.334868
1,288,rennes,5,1546.052632,4.45,1.57,0.08,0.00,0.09,1.01,...,58.20,56.48,44.05,23.63,10.44,10.24,8.72,309.210526,154.605263,104976.334868
2,4,montparnasse-bienvenue,5,67112.684211,5.71,0.26,0.01,0.04,0.24,3.15,...,55.50,62.72,49.65,31.05,21.54,16.83,13.49,13422.536842,6711.268421,104976.334868
3,4,montparnasse-bienvenue,5,67112.684211,8.99,1.58,0.08,0.00,0.00,2.24,...,40.52,44.32,41.17,32.13,24.22,21.47,19.70,13422.536842,6711.268421,104976.334868
4,27,madeleine,5,12904.421053,4.97,1.52,0.11,0.02,0.00,0.87,...,65.73,71.49,60.81,23.76,11.64,11.65,10.10,2580.884211,1290.442105,104976.334868
5,88,sevres-babylone,5,6540.105263,4.71,1.28,0.07,0.01,0.00,0.78,...,59.22,66.54,57.85,35.27,12.25,9.71,8.96,1308.021053,654.010526,104976.334868
6,115,mairie d'issy,5,6616.526316,2.55,0.89,0.06,0.00,0.05,4.60,...,38.97,38.91,30.74,16.37,8.48,6.66,5.20,1323.305263,661.652632,104976.334868
7,188,notre-dame-de-lorette,5,3949.578947,10.62,4.42,0.04,0.00,0.00,1.85,...,44.18,49.69,39.57,25.65,17.51,18.02,18.33,789.915789,394.957895,104976.334868
8,93,porte de versailles,5,15023.947368,12.16,4.87,0.09,0.01,0.00,1.58,...,60.08,62.00,53.19,32.02,19.18,19.01,18.49,3004.789474,1502.394737,104976.334868
9,136,vaugirard,5,4768.736842,4.38,1.34,0.07,0.00,0.00,1.95,...,44.22,41.04,34.89,19.16,10.52,9.18,7.66,953.747368,476.873684,104976.334868


Tableau 'ligne numéro[13]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,nb_correspondance,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,74,saint-denis-universite,5,8049.368421,1.95,0.42,0.01,0.00,0.00,18.07,...,31.51,27.77,18.64,13.60,8.89,6.17,4.36,1609.873684,804.936842,37332.821053
1,122,les courtilles,5,8152.789474,2.39,0.58,0.02,0.00,0.00,11.63,...,35.24,31.76,23.70,16.18,10.77,7.34,4.88,1630.557895,815.278947,37332.821053
2,125,porte de saint-ouen,5,6262.368421,5.48,2.05,0.05,0.00,0.00,4.58,...,42.89,40.38,34.53,23.47,16.62,12.24,9.27,1252.473684,626.236842,37332.821053
3,29,porte de clichy,5,12615.105263,9.95,3.26,0.27,0.06,0.65,11.07,...,92.43,88.85,71.86,46.94,32.62,24.38,18.66,2523.021053,1261.510526,37332.821053
4,33,place clichy,5,10922.736842,13.84,5.13,0.13,0.00,0.06,3.44,...,41.68,40.48,38.04,26.06,20.67,21.62,21.26,2184.547368,1092.273684,37332.821053
5,87,porte de vanves,5,7711.315789,4.61,1.30,0.07,0.01,0.00,5.18,...,41.18,39.19,30.95,21.34,13.95,10.18,8.11,1542.263158,771.131579,37332.821053
6,162,pernety,5,4404.263158,5.99,1.92,0.09,0.01,0.00,3.69,...,34.66,34.29,31.53,21.05,13.67,12.71,10.96,880.852632,440.426316,37332.821053
7,230,carrefour pleyel,5,3933.789474,3.40,1.46,0.10,0.00,0.05,4.69,...,57.68,45.30,28.87,16.22,10.19,8.39,6.78,786.757895,393.378947,37332.821053
8,64,gabriel peri,5,9837.842105,3.24,0.79,0.02,0.00,0.00,7.05,...,38.88,38.49,29.48,18.40,11.34,7.23,5.40,1967.568421,983.784211,37332.821053
9,69,miromesnil,5,8603.789474,8.08,2.75,0.05,0.01,0.05,1.50,...,60.25,65.41,47.45,26.32,13.69,14.06,11.90,1720.757895,860.378947,37332.821053


Tableau 'ligne numéro[14]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,nb_correspondance,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,3,gare de lyon,5,66037.947368,2.16,0.01,0.00,NaN,0.34,1.20,...,57.99,60.21,38.73,27.60,17.40,10.24,4.80,13207.589474,6603.794737,47415.801671
1,3,gare de lyon,5,66037.947368,18.38,4.31,0.30,0.10,0.62,8.89,...,84.89,89.26,74.77,56.04,41.78,38.76,34.89,13207.589474,6603.794737,47415.801671
2,61,bercy,5,9023.947368,NaN,NaN,NaN,NaN,NaN,6.98,...,104.32,67.23,40.18,13.57,0.11,14.23,0.09,1804.789474,902.394737,47415.801671
3,61,bercy,5,9023.947368,10.59,2.39,0.22,0.02,0.30,3.90,...,43.36,44.68,36.17,35.22,24.11,25.58,36.50,1804.789474,902.394737,47415.801671
4,27,madeleine,5,12904.421053,4.97,1.52,0.11,0.02,0.00,0.87,...,65.73,71.49,60.81,23.76,11.64,11.65,10.10,2580.884211,1290.442105,47415.801671
5,29,porte de clichy,5,12615.105263,9.95,3.26,0.27,0.06,0.65,11.07,...,92.43,88.85,71.86,46.94,32.62,24.38,18.66,2523.021053,1261.510526,47415.801671
6,110,cour saint-emilion,5,3173.444444,11.21,1.93,0.12,0.00,0.00,1.45,...,61.54,60.91,45.62,29.16,25.69,15.63,12.76,634.688889,317.344444,47415.801671
7,6,bibliotheque,5,6088.000000,6.46,1.35,0.06,0.00,0.10,5.02,...,54.02,56.83,44.83,26.99,18.84,11.58,8.76,1217.600000,608.800000,47415.801671
8,31,olympiades,5,12997.142857,2.74,1.23,0.06,0.00,0.01,6.71,...,47.26,41.98,31.78,20.50,13.19,6.90,4.47,2599.428571,1299.714286,47415.801671
9,39,mairie de saint-ouen,5,12017.736842,7.75,2.71,0.19,0.01,0.01,11.46,...,93.80,89.63,63.45,40.80,24.30,16.35,13.66,2403.547368,1201.773684,47415.801671
